In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Quickstart

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Ihsan")])

AIMessage(content='Hello Ihsan! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 14, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-86a23149-88d1-4f96-b562-83c76105462a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26})

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I don't have access to personal information about users.", response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 12, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5cbd1ac-04b5-4a1d-ad2a-fef9af38aee0-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Ihsan"),
        AIMessage(content="Hello Ihsan! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Ihsan. How can I assist you today, Ihsan?', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 39, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-07b480b8-c5f6-49d0-b9b6-e334b336c031-0', usage_metadata={'input_tokens': 39, 'output_tokens': 18, 'total_tokens': 57})

## Message History

In [12]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [13]:
config = {"configurable": {"session_id": "abc2"}}

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Ihsan")],
    config=config,
)

response.content

'Hello Ihsan! How can I assist you today?'

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Ihsan. How can I help you today?'

In [16]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I don't have the ability to know your name. If you would like me to refer to you by a specific name, please let me know."

In [17]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Ihsan. How can I assist you further?'

## Prompt Templates

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a joyful assistant with dad jokes ability. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Ihsan")]})

response.content

"Hi Ihsan! It's nice to meet you! How can I help you today?"

In [20]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [21]:
config = {"configurable": {"session_id": "abc5"}}

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Ican")],
    config=config,
)

response.content

"Hi Ican! I'm your joyful assistant here to help you with anything you need. How can I assist you today?"

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Ican! And I can help you with any questions or tasks you have.'

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a joyful assistant with dad jokes ability. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [25]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm Ihsan")], "language": "Indonesian"}
)

response.content

'Halo, Ihsan! Senang bertemu denganmu. Ada yang bisa saya bantu hari ini?'

In [26]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [27]:
config = {"configurable": {"session_id": "abc11"}}

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Ihsan")], "language": "Indonesian"},
    config=config,
)

response.content

'Halo, Ihsan! Selamat datang! Ada yang bisa saya bantu hari ini? 😊'

In [29]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Indonesian"},
    config=config,
)

response.content

'Nama kamu adalah Ihsan! Sudah pasti saya ingat, tidak perlu khawatir! 😄\n\n'

## Managing Conversation History

In [32]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [33]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to your personal information. What's your name?"

In [34]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2?"'

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to personal information."

In [37]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked me a math problem yet! But I'm ready whenever you are."

## Streaming

In [38]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Todd|!| Why| don|'t| scientists| trust| atoms|?| Because| they| make| up| everything|!| 😄||